In [9]:
import numpy as np
import pandas as pd
import scikit-learn

SyntaxError: invalid syntax (81543541.py, line 3)

In [10]:
pip install scikit-learn


     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     - -------------------------------------- 0.2/8.3 MB 4.8 MB/s eta 0:00:02
     -- ------------------------------------- 0.5/8.3 MB 4.8 MB/s eta 0:00:02
     -- ------------------------------------- 0.6/8.3 MB 4.8 MB/s eta 0:00:02
     --- ------------------------------------ 0.8/8.3 MB 4.1 MB/s eta 0:00:02
     ---- ----------------------------------- 1.0/8.3 MB 4.4 MB/s eta 0:00:02
     ----- ---------------------------------- 1.2/8.3 MB 4.3 MB/s eta 0:00:02
     ------- -------------------------------- 1.5/8.3 MB 4.5 MB/s eta 0:00:02
     -------- ------------------------------- 1.7/8.3 MB 4.6 MB/s eta 0:00:02
     --------- ------------------------------ 2.0/8.3 MB 4.6 MB/s eta 0:00:02
     ---------- ----------------------------- 2.1/8.3 MB 4.5 MB/s eta 0:00:02
     ---------- ----------------------------- 2.3/8.3 MB 4.4 MB/s eta 0:00:02
     ----------- ---------------------------- 2.5/8.3 MB 4.4 MB/s eta 0

In [ ]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
movies.info()
credits.info()

In [ ]:
movies.head(1)
credits.head(1)

In [ ]:
movies  = movies.merge(credits,on="title")

In [ ]:
movies.shape

In [ ]:
# not importanat
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
movies['genres'].head(1)

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [ ]:
import ast

In [ ]:
movies.isna().count()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['genres']

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
# movies['cast'][0]

In [ ]:
def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

In [ ]:
convert2(movies['cast'][0])

In [ ]:
movies['cast'] = movies['cast'].apply(convert2)

In [ ]:
# movies['crew'][0]

In [ ]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [ ]:
fetch_director(movies['crew'][0])

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head(1)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])


In [ ]:
movies.head(1)

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head(1)

In [ ]:
# movies['tags'][0]

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
stem("In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'")

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

In [5]:
pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
new_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000 ,stop_words = "english")

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
new_df[new_df['title'] == 'Batman Begins'].index[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x: x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x: x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])
    

In [ ]:
recommend('Avatar')

In [ ]:
print(new_df.iloc[1216]['title'])

In [ ]:
new_df.iloc[1216]['title']

In [ ]:
recommend('Batman')

In [ ]:
# recommend('superman')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict2121.pkl','wb'))

In [ ]:
# new_df.to_dict()

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))